 ## Глава 3: Низкоранговая адаптация (LoRA)

 ### Содержание главы

 В этой главе мы:



 *   Поймем, что такое низкоранговый адаптер и почему он полезен.

 *   Подготовим квантованную модель для обучения.

 *   Используем библиотеку `peft` для создания и подключения адаптеров к базовой модели.

 *   Обсудим параметры конфигурации для выбора целевых слоев при обучении.

 ### Настройка окружения

In [1]:
# Если вы используете Colab
!pip install datasets bitsandbytes trl


In [2]:
# Если вы используете Jupyter Template от runpod.io
#!pip install datasets bitsandbytes trl transformers peft huggingface-hub accelerate safetensors pandas matplotlib


 ### Импорт библиотек

In [3]:
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
from numpy.linalg import matrix_rank
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


 ### Цель использования адаптеров

 Мы подключаем адаптеры к огромным линейным слоям в LLM, чтобы резко сократить количество обучаемых параметров. Мы можем легко уменьшить количество обучаемых параметров до менее чем 1% от их исходного числа. За счет сокращения как вычислений (меньше градиентов для расчета), так и объема памяти (меньше параметров, отслеживаемых оптимизатором), мы достигаем значительного прироста эффективности. Однако следует помнить, что низкоранговые адаптеры вряд ли сравнятся по производительности с полной настройкой модели, и их эффективность может варьироваться в зависимости от базовой модели и задачи.

 ### Предварительные сведения

 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/matmul.png?raw=True)

 <center>Рисунок 3.1 - Умножение матриц</center>

 ### Суть низкоранговой адаптации

 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/two_matrices.png?raw=True)

 <center>Рисунок 3.2 - Умножение двух низкоранговых матриц</center>

In [4]:
base_layer = nn.Linear(1024, 1024, bias=False)
base_layer.weight.shape, base_layer.weight.numel()


(torch.Size([1024, 1024]), 1048576)

 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/lowrank_matrices.png?raw=True)

 <center>Рисунок 3.3 - Замороженные веса и низкоранговые матрицы</center>

In [5]:
torch.manual_seed(11)
r = 8
layer_A = nn.Linear(base_layer.in_features, r, bias=False)
layer_B = nn.Linear(r, base_layer.out_features, bias=False)
layer_A, layer_B


(Linear(in_features=1024, out_features=8, bias=False),
 Linear(in_features=8, out_features=1024, bias=False))

In [6]:
layer_A.weight.numel(), layer_B.weight.numel()


(8192, 8192)

In [7]:
composite = layer_B.weight @ layer_A.weight
composite.shape, composite.numel()


(torch.Size([1024, 1024]), 1048576)

In [8]:
matrix_rank(composite.detach().numpy())


np.int64(8)

 $$

 \Large

 \text{output} = X @ (W + B @ A)^T

 $$

 <center>Уравнение 3.1 - Добавление результирующего произведения к весам</center>

In [9]:
torch.manual_seed(19)
batch = torch.randn(1, 1024)

batch @ (base_layer.weight.data + layer_B.weight @ layer_A.weight).T


tensor([[-0.3909,  1.1422,  0.0897,  ..., -0.5231,  0.1517,  0.6949]],
       grad_fn=<MmBackward0>)

 $$

 \Large

 \text{output} = \underbrace{X @ W^T}_{O_W} + \underbrace{X @ (B @ A)^T}_{O_{AB}}

 $$

 <center>Уравнение 3.2 - Использование двух прямых проходов</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/forward.png?raw=True)

 <center>Рисунок 3.4 - Использование двух прямых проходов</center>

In [10]:
regular_output = batch @ base_layer.weight.data.T
additional_output = batch @ (layer_B.weight @ layer_A.weight).T
regular_output, additional_output


(tensor([[-0.3552,  1.1192,  0.5504,  ..., -0.2310, -0.0427,  0.1908]]),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>))

 $$

 \Large

 \text{additional} = X @ (B @ A)^T = \underbrace{\underbrace{(X @ A^T)}_{O_A} @ B^T}_{O_{AB}}

 $$

 <center>Уравнение 3.3 - Цепочка прямых проходов адаптера</center>

In [11]:
out_A = (batch @ layer_A.weight.T)
additional_output = out_A @ layer_B.weight.T
additional_output


tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
       grad_fn=<MmBackward0>)

In [12]:
regular_output = base_layer(batch)
out_A = layer_A(batch)
additional_output = layer_B(out_A)
output = regular_output + additional_output
regular_output, additional_output, output


(tensor([[-0.3552,  1.1192,  0.5504,  ..., -0.2310, -0.0427,  0.1908]],
        grad_fn=<MmBackward0>),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>),
 tensor([[-0.3909,  1.1422,  0.0897,  ..., -0.5231,  0.1517,  0.6949]],
        grad_fn=<AddBackward0>))

 $$

 \Large

 \text{output} = X @ W^T + \frac{\alpha}{r}\left[X @ (B @ A)^T\right]

 $$

 <center>Уравнение 3.4 - Параметр alpha в LoRA</center>

In [13]:
alpha = 2*r
output = regular_output + (alpha / r) * additional_output
output


tensor([[-0.4267,  1.1652, -0.3710,  ..., -0.8151,  0.3461,  1.1989]],
       grad_fn=<AddBackward0>)

 ### Итог предыдущих глав

In [14]:
supported = torch.cuda.is_bf16_supported(including_emulation=False)
compute_dtype = (torch.bfloat16 if supported else torch.float32)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=compute_dtype
)

from huggingface_hub import snapshot_download

# Download entire repository
snapshot_download(
    repo_id="facebook/opt-350m",
    local_dir="./opt-350m-model",
    ignore_patterns=["*.h5", "*.ot", "*.msgpack"],  # Skip unnecessary formats
    local_dir_use_symlinks=False
)

# Load from local directory
model_dir = "./opt-350m-model"


model_q4 = AutoModelForCausalLM.from_pretrained(model_dir,
                                                device_map='cuda:0',
                                                torch_dtype=compute_dtype,
                                                quantization_config=nf4_config)


/home/loschilov_aa/.conda/envs/pytorchbook/lib/python3.10/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


 ### Типы параметров и градиенты

 ****

 **Итог раздела "Типы параметров и градиенты"**

 *   Квантование замораживает только те линейные слои, которые были квантованы.

 *   После квантования модель можно подготовить с помощью функции `prepare_model_for_kbit_training()`.

     *   Она замораживает **все** слои.

     *   Она преобразует каждый неквантованный 16-битный слой в FP32 для улучшения обучения.

     *   Она включает контрольные точки градиента (gradient checkpointing).

 *   Позже вы сможете разморозить выбранные слои с помощью конфигурации LoRA.

 ****

In [15]:
def trainable_parms(model):
    parms = [(name, param.dtype) for name, param in model.named_parameters() if param.requires_grad]
    return parms

trainable_parms(model_q4.model)


[('decoder.embed_tokens.weight', torch.bfloat16),
 ('decoder.embed_positions.weight', torch.bfloat16),
 ('decoder.layers.0.self_attn_layer_norm.weight', torch.bfloat16),
 ('decoder.layers.0.self_attn_layer_norm.bias', torch.bfloat16),
 ('decoder.layers.0.final_layer_norm.weight', torch.bfloat16),
 ('decoder.layers.0.final_layer_norm.bias', torch.bfloat16),
 ('decoder.layers.1.self_attn_layer_norm.weight', torch.bfloat16),
 ('decoder.layers.1.self_attn_layer_norm.bias', torch.bfloat16),
 ('decoder.layers.1.final_layer_norm.weight', torch.bfloat16),
 ('decoder.layers.1.final_layer_norm.bias', torch.bfloat16),
 ('decoder.layers.2.self_attn_layer_norm.weight', torch.bfloat16),
 ('decoder.layers.2.self_attn_layer_norm.bias', torch.bfloat16),
 ('decoder.layers.2.final_layer_norm.weight', torch.bfloat16),
 ('decoder.layers.2.final_layer_norm.bias', torch.bfloat16),
 ('decoder.layers.3.self_attn_layer_norm.weight', torch.bfloat16),
 ('decoder.layers.3.self_attn_layer_norm.bias', torch.bfloat16

 #### Функция `prepare_model_for_kbit_training()`

In [16]:
prepared_model = prepare_model_for_kbit_training(model_q4,
                                        use_gradient_checkpointing=True,
                                        gradient_checkpointing_kwargs={'use_reentrant': False})
prepared_model


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (

In [17]:
trainable_parms(prepared_model)


[]

In [18]:
def parms_of_dtype(model, dtype=torch.float32):
    parms = [name for name, param in model.named_parameters() if param.dtype == dtype]
    return parms


In [19]:
parms_of_dtype(prepared_model)


['model.decoder.embed_tokens.weight',
 'model.decoder.embed_positions.weight',
 'model.decoder.layers.0.self_attn.k_proj.bias',
 'model.decoder.layers.0.self_attn.v_proj.bias',
 'model.decoder.layers.0.self_attn.q_proj.bias',
 'model.decoder.layers.0.self_attn.out_proj.bias',
 'model.decoder.layers.0.self_attn_layer_norm.weight',
 'model.decoder.layers.0.self_attn_layer_norm.bias',
 'model.decoder.layers.0.fc1.bias',
 'model.decoder.layers.0.fc2.bias',
 'model.decoder.layers.0.final_layer_norm.weight',
 'model.decoder.layers.0.final_layer_norm.bias',
 'model.decoder.layers.1.self_attn.k_proj.bias',
 'model.decoder.layers.1.self_attn.v_proj.bias',
 'model.decoder.layers.1.self_attn.q_proj.bias',
 'model.decoder.layers.1.self_attn.out_proj.bias',
 'model.decoder.layers.1.self_attn_layer_norm.weight',
 'model.decoder.layers.1.self_attn_layer_norm.bias',
 'model.decoder.layers.1.fc1.bias',
 'model.decoder.layers.1.fc2.bias',
 'model.decoder.layers.1.final_layer_norm.weight',
 'model.decode

In [20]:
prepared_model.get_memory_footprint()/1e6


264.15104

 ### Библиотека PEFT

 "*🤗 PEFT (Parameter-Efficient Fine-Tuning) — это библиотека для эффективной адаптации больших предварительно обученных моделей к различным прикладным задачам без тонкой настройки всех параметров модели, поскольку это непомерно дорого. Методы PEFT настраивают только небольшое количество (дополнительных) параметров модели, значительно снижая вычислительные затраты и затраты на хранение, при этом обеспечивая производительность, сравнимую с полностью настроенной моделью. Это делает обучение и хранение больших языковых моделей (LLM) на потребительском оборудовании более доступным.*



 *PEFT интегрирована с библиотеками Transformers, Diffusers и Accelerate, чтобы обеспечить более быстрый и простой способ загрузки, обучения и использования больших моделей для вывода.*"



 ****

 **Итог раздела "PEFT"**

 *   Базовая конфигурация ниже должна хорошо работать во многих случаях.

 ```python

 config = LoraConfig(

     r=16,

     lora_alpha=32,

     lora_dropout=0.05,

     bias="none",

     task_type="CAUSAL_LM",

 )

 peft_model = get_peft_model(model, config)

 ```

 *   Ранги 8, 16 или 32 являются типичными, но использование более высоких значений не должно значительно влиять на объем памяти модели.

 *   Масштабирующий коэффициент `lora_alpha` обычно в два раза больше ранга.

 *   Если ваша модель имеет слои `Conv1D`, добавьте `fan_in_fan_out=True` в конфигурацию.

 *   Если ваша модель была выпущена недавно, вам может потребоваться вручную указать `target_modules`.

     *   Обычно используются имена массивных линейных слоев в модуле внимания.

 *   По умолчанию обучаемыми являются только адаптеры.

     *   Если вы хотите обучать другие слои, такие как слои нормализации, добавьте их в аргумент `modules_to_save`.

     *   Если вы добавляете собственные токены в токенизатор, вам также нужно будет обучить слои, связанные со словарем, такие как эмбеддинги и голову модели.

 ****

In [21]:
lora_config = LoraConfig()
lora_config


LoraConfig(task_type=None, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules=None, exclude_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [22]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


 ### Параметр `target_modules`

 Поскольку новые модели и архитектуры выпускаются еженедельно, вполне вероятно, что в установленной версии библиотеки PEFT нет предварительно настроенного списка целевых слоев. В этом случае вы столкнетесь со следующей ошибкой:



 ***

 `ValueError: Please specify `target_modules` in `peft_config``

 ***



 Как только вы получите имена, вы можете использовать еще один аргумент конфигурации: `target_modules`, который представляет собой имя или список имен модулей, к которым вы хотите применить адаптеры.



 **Поддерживаемые модели**



 Если вы хотите проверить, поддерживается ли архитектура данной модели установленной версией пакета `peft`, вы можете сделать следующее:

In [23]:
from peft.utils.constants import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING.keys()


dict_keys(['t5', 'mt5', 'bart', 'gpt2', 'bloom', 'blip-2', 'opt', 'gptj', 'gpt_neox', 'gpt_neo', 'bert', 'roberta', 'xlm-roberta', 'electra', 'deberta-v2', 'deberta', 'layoutlm', 'llama', 'llama4', 'chatglm', 'gpt_bigcode', 'mpt', 'RefinedWebModel', 'RefinedWeb', 'falcon', 'btlm', 'codegen', 'mistral', 'mixtral', 'stablelm', 'phi', 'gemma', 'gemma2', 'gemma3_text', 'qwen2', 'qwen3'])

In [24]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['phi']


['q_proj', 'v_proj', 'fc1', 'fc2']

 #### PEFT-модель

In [25]:
peft_model = get_peft_model(prepared_model, config, adapter_name='default')
peft_model


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
          (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_fea

In [26]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['opt']


['q_proj', 'v_proj']

In [27]:
lin = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj
lin


lora.Linear4bit(
  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=1024, out_features=8, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=8, out_features=1024, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)

In [28]:
peft_model.print_trainable_parameters()


trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


In [29]:
trainable_parms(peft_model.base_model.model)


[('model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_B.default.weight'

 #### Параметр `modules_to_save`

In [30]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm']
)


In [31]:
# Поскольку модель изменяется на месте, нам необходимо выгрузить адаптеры
# из предыдущей конфигурации, чтобы избежать их смешивания.
# В обычном рабочем процессе вы загружаете конфигурацию только один раз,
# и это не потребуется.
_ = peft_model.unload()


In [32]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()


trainable params: 884,736 || all params: 332,081,152 || trainable%: 0.2664


/home/loschilov_aa/.conda/envs/pytorchbook/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


 #### Эмбеддинги

In [33]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm', 'embed_tokens']
)


In [34]:
# Поскольку модель изменяется на месте, нам необходимо выгрузить адаптеры
# из предыдущей конфигурации, чтобы избежать их смешивания.
# В обычном рабочем процессе вы загружаете конфигурацию только один раз,
# и это не потребуется.
_ = peft_model.unload()


In [35]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()


trainable params: 26,624,000 || all params: 357,820,416 || trainable%: 7.4406


In [36]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['embed_tokens', 'q_proj', 'v_proj']
)


In [37]:
# Поскольку модель изменяется на месте, нам необходимо выгрузить адаптеры
# из предыдущей конфигурации, чтобы избежать их смешивания.
# В обычном рабочем процессе вы загружаете конфигурацию только один раз,
# и это не потребуется.
_ = peft_model.unload()


In [38]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()


trainable params: 1,192,704 || all params: 358,128,384 || trainable%: 0.3330


/home/loschilov_aa/.conda/envs/pytorchbook/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.decoder.embed_tokens'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [39]:
lin = peft_model.base_model.model.model.decoder.embed_tokens
lin


lora.Embedding(
  (base_layer): Embedding(50272, 512, padding_idx=1)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict()
  (lora_B): ModuleDict()
  (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 8x50272 (cuda:0)])
  (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 512x8 (cuda:0)])
  (lora_magnitude_vector): ModuleDict()
)

 #### Управление адаптерами

In [40]:
adapter_dir = "./opt-350m-lora-yoda"
peft_model.load_adapter(adapter_dir, adapter_name='yoda')
lora_A = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj.lora_A
lora_A


ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [41]:
peft_model.add_adapter(adapter_name='third', peft_config=config)
lora_A


/home/loschilov_aa/.conda/envs/pytorchbook/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.decoder.embed_tokens', 'model.decoder.embed_tokens'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
  (third): Linear(in_features=1024, out_features=8, bias=False)
)

In [42]:
peft_model.delete_adapter(adapter_name='third')
lora_A


ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [43]:
peft_model.peft_config.keys()


dict_keys(['default', 'yoda'])

In [44]:
peft_model.active_adapter


'default'

In [45]:
peft_model.set_adapter('yoda')
peft_model.active_adapter


'yoda'

 ```python

 with peft_model.disable_adapter():

     original_outputs = peft_model(inputs)



 original_outputs = peft_model.base_model(inputs)

 ```

In [46]:
peft_model.merge_adapter(adapter_names=['yoda'])
lora_A


/home/loschilov_aa/.conda/envs/pytorchbook/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:424: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.decoder.embed_tokens', 'model.decoder.embed_tokens'] are part of the adapter. This can lead to complications. You can opt to merge the adapter after cloning the weights (to untie the embeddings). You can untie the embeddings by loading the model with `tie_word_embeddings=False`. For example:
```python
from transformers import AutoModelForCausalLM

# Load original tied model
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", tie_word_embeddings=False)

# Set the randomly initialized lm_head to the previously tied embeddings
model.lm_head.weight.data = model.model.embed_tokens.weight.data.clone()

# Save the untied model
untied_model_dir = "dir/for/untied/model"
model.save_pretrained(untied_model_dir)
model.config.save_pretrained(untied_model_dir)

# Now use the original model but 

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [47]:
peft_model.unload()
peft_model.base_model.model.model.decoder.layers[0].self_attn


OPTAttention(
  (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
)

 ### Что ждет в книге "Fine-Tuning LLMs"

 Низкоранговые адаптеры спасли положение, позволив быструю и дешевую тонкую настройку LLM. Эти огромные модели, хотя и мощные, являются мастерами одного дела — предсказания следующего токена — и поэтому остаются ограниченными структурой своих входных данных. Чтобы эти существа могли общаться, необходимо разработать новый вид ввода. Узнайте больше об удивительной истории шаблонов чата в следующей главе "Fine-Tuning LLMs".